# <div align="center"> Part5 : Live streaming and processing the twitter data </div>

## Twitter Stream API

The Streaming APIs give developers low latency access to Twitter’s global stream of Tweet data. A proper implementation of a streaming client will be pushed messages indicating Tweets and other events have occurred, without any of the overhead associated with polling a REST endpoint.

Connecting to the streaming API requires keeping a persistent HTTP connection open. In many cases this involves thinking about your application differently than if you were interacting with the REST API.

## Tweepy API

tweepy is a Twitter API wrapper. You can access most of twitter REST and Stream API using Tweepy. In our project, we use Tweepy to access the twitter stream data. 

## Python Server Program

1.config twitter API, assign access_token, access_secret, consumer_key, consumer_secret

In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json


#config twitter API credential here
access_token = "721369358241386496-9yGA7KdVhJFgYUKmT1QRKIhHaa9h5li"
access_secret = "R63VZJtUCioxPSEbg41FzTxORke0uY5sX0UQyAeiadJ3x"
consumer_key = "jOruEScM0dpmt5DKuh7fqL4IB"
consumer_secret = "oLpHlVjRx9lV1cBncJJlPmH0NInA7rscR6ndULA7IiDwoXcJFN"

2.define a tweetsListener, we need a python server program to read twitter stream from twitter and write the twitter stream into the socket. We create a socket binds to localhost port 5555, wait for the client to accept. Write twitter stream into the client as long as it accepts. 

In [ ]:
     class TweetsListener(StreamListener):

  def __init__(self, csocket):
      self.client_socket = csocket

  def on_data(self, data):
      try:
          msg = json.loads( data )
          print (msg['created_at'],msg['user']['name'],msg['text'])
          #print( msg['text'].encode('utf-8') )
          self.client_socket.send( msg['text'].encode('utf-8') )
          return True
      except BaseException as e:
          print("Error on_data: %s" % str(e))
      except:
          self.sock.close()
      return True
      

  def on_error(self, status):
      print(status)
      return True

def sendData(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)

  twitter_stream = Stream(auth, TweetsListener(c_socket))
  twitter_stream.filter(track=['@ebay','@amazon','@Target','@Walmart','@BestBuy'])

if __name__ == "__main__":
  s = socket.socket()         # Create a socket object
  host = "localhost"      # Get local machine name
  port = 5555                 # Reserve a port for your service.
  s.bind((host, port))        # Bind to the port

  print("Listening on port: %s" % str(port))

  s.listen(5)                 # Now wait for client connection.
  c, addr = s.accept()        # Establish connection with client.

  print( "Received request from: " + str( addr ) )

  sendData( c )

## spark client program

Next steps is to start the spark program as a client program to get twitter stream from the socket.

1.we need to import library and set environment path.

In [3]:
#Importing Library and setting environment path
import os
import sys
#set the path 
os.environ['SPARK_HOME'] = "C:/spark-2.0.2-bin-hadoop2.7"
sys.path.append("C:/park-2.0.2-bin-hadoop2.7/bin")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark/lib")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark/lib/pyspark.zip")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark/lib/py4j-0.10.3-src.zip")
sys.path.append("C:/Program Files/Java/jre1.8.0_77/bin")

2.we should import all the module we need to our python spark program

3.get the sparkContext.

In [ ]:
sc = SparkContext()

4.set the streaming context.

In [ ]:
ssc = StreamingContext(sc, 10 )

5.get the sqlContext.

In [ ]:
sqlContext = SQLContext(sc)

6.set streaming context checkpointing.

In [ ]:
ssc.checkpoint( "file:///checkpoint")

7.create a Dstream from the socket we create from the server python program

In [ ]:
socket_stream = ssc.socketTextStream("localhost", 5555)

8.perform window operation to the dstream

In [ ]:
lines = socket_stream.window( 20 )

9.give names to the fields

In [11]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

10.perform MapReduce transformation to lines, register a temp table tweets.

In [ ]:
(lines.flatMap( lambda text: text.split( " " ) )
  .filter( lambda word: word.lower().startswith("@") )
  .map( lambda word: ( word.lower(), 1 ) )
  .reduceByKey( lambda a, b: a + b )
  .map( lambda rec: Tweet( rec[0], rec[1] ) ).foreachRDD(lambda rdd: rdd.toDF()
  .sort(desc("count")).limit(10).registerTempTable("tweets"))
   )

11.start the stream

In [ ]:
ssc.start()

12.do sql qeury to the tweets table

In [ ]:
top_10_tweets = sqlContext.sql( 'select * from tweets limit 10' )
top_10_tweets.head(5)

13.import the pyplot 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

14.plot tag with count.

In [ ]:
import time
from IPython import display


count = 0
while count < 10:
  time.sleep( 20 )
  top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
  top_10_df = top_10_tweets.toPandas()
  display.clear_output(wait=True)
  sn.plt.figure( figsize = ( 10, 8 ) )
  sn.barplot( x="count", y="tag", data=top_10_df)
  sn.plt.show()
  count = count + 1

15.stop the streaming

In [ ]:
ssc.stop(False)